In [1]:
import numpy as np

In [3]:
def score(c1, c2):
    if c1 == c2: return  1
    else: return -1

def needleman_wunsch(seq1, seq2):
    mat = np.zeros((len(seq1) + 1, len(seq2) + 1))
    mat[:, 0] = -np.arange(len(seq1) + 1)
    mat[0, :] = -np.arange(len(seq2) + 1)

    mat_arrows = np.zeros_like(mat)

    for i in range(1, len(seq1) + 1):
        for j in range(1, len(seq2) + 1):
            top = mat[i-1, j] - 1
            left = mat[i, j-1] - 1
            top_left = mat[i-1, j-1] +  score(seq1[i-1], seq2[j-1])

            mat[i,j] = max([left, top_left, top])

            mat_arrows[i,j] = np.argmax([left, top_left, top])


    return mat, mat_arrows

In [55]:
def process_arrows(mat_arrows, seq1, seq2):
    i, j = mat_arrows.shape[0] - 1, mat_arrows.shape[1] - 1

    res_seq1 = []
    res_seq2 = []

    while i != 0 and j != 0:
        print(i, j, res_seq1, res_seq2)
        if mat_arrows[i, j] == 1:
            res_seq1.append(seq1[i-1])
            res_seq2.append(seq2[j-1])
            i -= 1
            j -= 1
        elif mat_arrows[i, j] == 0:
            res_seq1.append('-')
            res_seq2.append(seq2[j - 1])
            j -= 1
        elif mat_arrows[i, j] == 2:
            res_seq1.append(seq1[i - 1])
            res_seq2.append('-')
            i -= 1
    
    if i != 0:
        res_seq1.extend(seq1[:i])
        res_seq2.extend(['-'] * len(seq2[:i]))

    return res_seq1[::-1], res_seq2[::-1]#''.join(res_seq1[::-1]), ''.join(res_seq2[::-1])

In [56]:
s = 'AGTCATC'
t = 'ACGTAGC'

In [108]:
m, a= needleman_wunsch([0,1,2,3,4],[2,3,4])

In [109]:
m

array([[ 0., -1., -2., -3.],
       [-1., -1., -2., -3.],
       [-2., -2., -2., -3.],
       [-3., -1., -2., -3.],
       [-4., -2.,  0., -1.],
       [-5., -3., -1.,  1.]])

In [99]:
s1,s2= process_arrows(a, [0,1,2,3,4],[2,3,4])

5 3 [] []
4 2 [4] [4]
3 1 [4, 3] [4, 3]


In [100]:
m

array([[ 0., -1., -2., -3.],
       [-1., -1., -2., -3.],
       [-2., -2., -2., -3.],
       [-3., -1., -2., -3.],
       [-4., -2.,  0., -1.],
       [-5., -3., -1.,  1.]])

In [74]:
print(s1)
print(s2)

[1, 0, 2, 3, 4]
['-', '-', 2, 3, 4]


In [177]:
def preprint_seq(seq, max_len = 2):
    seq_str = [str(x) for x in seq]
    
    seq_str_allign = [' '*(max_len - len(x))+x for x in seq_str] 
    
    return '[' + '  '.join(seq_str_allign)+"]"

In [178]:
print(preprint_seq([1, 0, 2, 3, 4, 5]))
print(preprint_seq(s2) + preprint_seq([5]))

[ 1   0   2   3   4   5]
[ -   -   2   3   4][ 5]


In [179]:
match_award      = 10
mismatch_penalty = -5
gap_penalty      = -10 # both for opening and extanding

def match_score(alpha, beta):
    if alpha == beta:
        return match_award
    elif alpha == '-' or beta == '-':
        return gap_penalty
    else:
        return mismatch_penalty


In [186]:
def water(seq1, seq2):
    m, n = len(seq1), len(seq2)  # length of two sequences
    
    # Generate DP table and traceback path pointer matrix
    score = zeros((m+1, n+1))      # the DP table
    pointer = zeros((m+1, n+1))    # to store the traceback path
    
    max_score = 0        # initial maximum score in DP table
    # Calculate DP table and mark pointers
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            score_diagonal = score[i-1][j-1] + match_score(seq1[i-1], seq2[j-1])
            score_up = score[i][j-1] + gap_penalty
            score_left = score[i-1][j] + gap_penalty
            score[i][j] = max(0,score_left, score_up, score_diagonal)
            if score[i][j] == 0:
                pointer[i][j] = 0 # 0 means end of the path
            if score[i][j] == score_left:
                pointer[i][j] = 1 # 1 means trace up
            if score[i][j] == score_up:
                pointer[i][j] = 2 # 2 means trace left
            if score[i][j] == score_diagonal:
                pointer[i][j] = 3 # 3 means trace diagonal
            if score[i][j] >= max_score:
                max_i = i
                max_j = j
                max_score = score[i][j];
    
    align1, align2 = [], []    # initial sequences
    
    i,j = max_i,max_j    # indices of path starting point
    
    #traceback, follow pointers
    while pointer[i][j] != 0:
        if pointer[i][j] == 3:
            align1.append(seq1[i-1])
            align2.append(seq2[j-1])
            i -= 1
            j -= 1
        elif pointer[i][j] == 2:
            align1.append('-')
            align2.append(seq2[j-1])
            j -= 1
        elif pointer[i][j] == 1:
            align1.append(seq1[i-1])
            align2.append('-')
            i -= 1

    return seq1, seq2, align1[::-1], align2[::-1], i, max_i, j, max_j, (max_i - i) / len(seq1), (max_j - j) / len(seq2), max_score
    #finalize(align1, align2)

In [156]:
water('AAACCCCGGTTT', 'CCCGAGA')

(['C', 'C', 'C', 'G', '-', 'G'],
 ['C', 'C', 'C', 'G', 'A', 'G'],
 4,
 9,
 0,
 6,
 0.4166666666666667,
 0.8571428571428571,
 40)

In [157]:
'AAACCCCGGTTT'[4:9]

'CCCGG'

In [158]:
water([1,2,3], [1,2, 4, 3])

([1, 2, '-', 3], [1, 2, 4, 3], 0, 3, 0, 4, 1.0, 1.0, 20)

In [159]:
r = [[23, 24, 16, 18, 19, 20, 17, 13, 14, 15, 25, 22, 26, 27],
                            [40, 39, 29, 30, 42],
                            [23, 94, 96, 101, 110, 130],
                            [23, 145, 149, 148, 153, 152, 177, 169, 200],
                            [31, 35, 49, 56, 50, 48, 58, 73, 76],
                            [80, 165, 174, 191],
                            [33, 36, 32],
                            [38, 34, 41, 45],
                            [46, 37, 28, 21, 10, 5, 11, 0, 1, 4, 2, 6, 3, 8, 12, 7, 9],
                            [70, 90, 69, 71, 68, 78, 89, 75, 83, 98],
                            [70, 118, 157, 150, 146, 166],
                            [85, 109, 112, 143, 144, 141, 139, 155, 170, 180],
                            [105, 66, 57, 43, 44, 47, 72],
                            [123, 99, 104, 106, 100, 108, 114, 124],
                            [127, 79, 67, 77, 81, 64, 55, 62, 60, 54, 59, 51, 53, 61, 65, 63, 74, 87, 92],
                            [197, 163, 95, 93, 82, 84, 91, 115, 117]]

In [160]:
a  = {'first_2': [197, 163, 95, 93],
 'first_3': [95, 93, 82, 84, 91, 115, 117],
 'first_5': [86, 31, 35, 49, 56, 50, 48, 58, 73, 76, 80],
 'first_7': [33, 36, 32, 25, 40, 39, 29, 30, 42],
 'first_8': [86, 31, 35, 50, 48, 58, 73, 76, 80, 165, 174, 191],
 'first_11': [127, 79, 67, 81, 64, 59, 51],
 'first_14': [46, 37, 28, 21, 10, 5, 11, 0, 1, 4, 2, 6, 3, 8, 12, 7, 9],
 'first_15': [127, 79, 67,  77, 81, 64, 55, 62, 60, 54, 59, 51, 51, 53, 61, 65, 63, 74, 87, 92],
 'first_21': [85, 109, 112, 143, 144, 141, 139, 155, 170, 180, 21],
 'first_22': [46, 37, 28, 21, 10, 5, 1, 3],
 'second_2': [23, 145, 149, 148, 153, 152, 177, 169, 200],
 'second_3': [123, 99, 104, 106, 100, 108, 114, 124],
 'second_5': [66, 57, 43, 44, 47, 72],
 'second_7': [105, 88, 97, 38, 34, 41, 45],
 'second_8': [66, 57, 43, 44, 47, 72],
 'second_11': [23, 96, 101, 110, 110],
 'second_14': [23, 24, 16, 18, 19, 20, 17, 13, 14, 15, 25, 22, 26, 27],
 'second_15': [23, 94, 96, 101, 110, 130],
 'second_21': [105, 66, 88, 97, 70, 118, 157, 150, 146, 166],
 'second_22': [90, 69, 71, 68, 78, 89, 75, 83, 98]}

In [161]:
a = [x for x in a.values()]

In [197]:
def best_alignments(array, arrays_set):
    alignments = np.array([water(array, x) for x in arrays_set])
    mask = np.array([x[-1] for x in alignments]) > 10
    return alignments[mask]

In [200]:
a[1]

[95, 93, 82, 84, 91, 115, 117]

In [202]:
def recalc_borders(alignment, seq, seq_i, seq_j, alg_i, alg_j):
    return seq_i - alg_i, (len(alignment) - alg_j) + seq_j

In [264]:
def preprint_seq(seq, min_border, max_border, start, end, cell_size = 4):
    seq_str = ['-']* (start - min_border) + [str(x) for x in seq] + ['-'] * (max_border - end)
    #print(seq_str)
    return ''.join([' ' + el + ' ' * (cell_size - len(el) - 1) for el in seq_str])
    #return '[' + '  '.join(seq_str_allign)+"]"

In [265]:
print(preprint_seq([1,2,3,4], -1, 10, 0, 4))
print(preprint_seq([2,'-',4], -1, 10, 1, 4))

 -   1   2   3   4   -   -   -   -   -   -  
 -   -   2   -   4   -   -   -   -   -   -  


In [266]:
preprint_seq([1,2,3,4], -1, 10, 0, 4)

' -   1   2   3   4   -   -   -   -   -   -  '

In [267]:
def print_alignments(alignments, seq, cell_size = 4):
    #print([[align[1], seq, align[4], align[5], align[6], align[7]] for align in alignments])
    alignments_idxes = [recalc_borders(align[1], seq, align[4], align[5], align[6], align[7]) for align in alignments]
    
    min_border = min([x[0] for x in alignments_idxes] + [0])
    max_border = max([x[0] for x in alignments_idxes] + [len(seq)])

    
    printable = []
    printable.append(preprint_seq(seq, min_border, max_border, 0, len(seq)))
    for align, idx in zip(alignments, alignments_idxes):
        printable.append(preprint_seq(align[1], min_border, max_border, idx[0], idx[1]))
    
    return printable

In [268]:
best_alignments(a[3], r)

array([[list([33, 36, 32, 25, 40, 39, 29, 30, 42]),
        list([40, 39, 29, 30, 42]), list([40, 39, 29, 30, 42]),
        list([40, 39, 29, 30, 42]), 4, 9, 0, 5, 0.5555555555555556, 1.0,
        50],
       [list([33, 36, 32, 25, 40, 39, 29, 30, 42]), list([33, 36, 32]),
        list([33, 36, 32]), list([33, 36, 32]), 0, 3, 0, 3,
        0.3333333333333333, 1.0, 30]], dtype=object)

In [269]:
print_alignments(best_alignments(a[3], r), a[3])

[' 33  36  32  25  40  39  29  30  42 ',
 ' -   -   -   -   40  39  29  30  42 ',
 ' 33  36  32  -   -   -   -   -   -  ']

In [270]:
seq = [1,2,3,4,5,6,7,8,9]

res = [[0,1,2,102,4],
       [5,6,7,8],
       [7,10,8,9,11,12]]
    

In [272]:
best_alignments(seq, res)

array([[list([1, 2, 3, 4, 5, 6, 7, 8, 9]), list([0, 1, 2, 102, 4]),
        list([1, 2, 3, 4]), list([1, 2, 102, 4]), 0, 4, 1, 5,
        0.4444444444444444, 0.8, 25],
       [list([1, 2, 3, 4, 5, 6, 7, 8, 9]), list([5, 6, 7, 8]),
        list([5, 6, 7, 8]), list([5, 6, 7, 8]), 4, 8, 0, 4,
        0.4444444444444444, 1.0, 40],
       [list([1, 2, 3, 4, 5, 6, 7, 8, 9]), list([7, 10, 8, 9, 11, 12]),
        list([7, '-', 8, 9]), list([7, 10, 8, 9]), 6, 9, 0, 4,
        0.3333333333333333, 0.6666666666666666, 20]], dtype=object)

In [271]:
print_alignments(best_alignments(seq, res), seq)

[' -   1   2   3   4   5   6   7   8   9  ',
 ' 0   1   2   102 4   -   -   -   -   -  ',
 ' -   -   -   -   -   5   6   7   8   -  ',
 ' -   -   -   -   -   -   -   7   10  8   9   11  12 ']